BDLE 2023 TP2

révision: 19/09/2023

BOUCHOUCHI Nour (28600211)  
BORCHANI Sofia (21212080)

# TP2 2023 Fenetre Foursquare avec le système DuckDB (ETUDIANT)


# Préparation

In [2]:
import pandas as pd
from google.colab import data_table

def display(df, n=100):
  return data_table.DataTable(df.limit(n).df(), include_index=False, num_rows_per_page=10)

In [3]:
!pip install duckdb==0.8.1

# Démarrer le service DuckDB

In [4]:
import duckdb

# db = duckdb.connect('foursquare')
db = duckdb.connect(':memory:')

# vérifier que le service fonctrionne
r = db.sql("SELECT 'hello' as col")
display(r)

,col
0,hello


# Données

In [5]:
import os
local_dir = "/local/data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

[]

URL pour l'accès aux datasets

In [6]:
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4"
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL pour les datasets ", PUBLIC_DATASET_URL)

URL pour les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


In [7]:
import os
from urllib import request
import zipfile

# download dataset if not already donwloaded
def download_file(web_dir, local_dir, file):
  local_file = local_dir + "/" + file
  web_file = web_dir + "/" + file
  if(os.path.isfile(local_file)):
    print(file, "is already stored")
  else:
    print("downloading from URL: ", web_file , "save in : " + local_file)
    request.urlretrieve(web_file , local_file)

def unzip_file(local_dir, file):
  with zipfile.ZipFile(local_dir + "/" + file, 'r') as zip_ref:
    zip_ref.extractall(local_dir)
  # os.remove(local_dir + "/" + file)


web_dir = PUBLIC_DATASET + "/foursquare"

# ce fichier vient de kaggle : https://www.kaggle.com/datasets/chetanism/foursquare-nyc-and-tokyo-checkin-dataset
zip_filename = "dataset_TSMC2014_NYC.zip"


download_file(web_dir, local_dir, zip_filename)
unzip_file(local_dir, zip_filename)

# Liste des fichiers
os.listdir(local_dir)

downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/foursquare/dataset_TSMC2014_NYC.zip save in : /local/data/dataset_TSMC2014_NYC.zip


['dataset_TSMC2014_NYC.zip', 'dataset_TSMC2014_NYC.csv']

In [8]:
!head -4 /local/data/dataset_TSMC2014_NYC.csv

userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp
470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.71981038,-74.00258103,-240,Tue Apr 03 18:00:09 +0000 2012
979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.60679958,-74.04416981,-240,Tue Apr 03 18:00:25 +0000 2012
69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.71616168,-73.88307006,-240,Tue Apr 03 18:02:24 +0000 2012


# Table Visit

In [9]:
# duckdb.read_csv('/local/data/notes1M.json')

stmt = """
DROP TABLE if exists Visit;
CREATE TABLE Visit as
  SELECT *
  FROM '/local/data/dataset_TSMC2014_NYC.csv';
"""
db.sql(stmt)
print("done")

done


## Afficher le contenu d'une table

In [10]:
visit = db.table('Visit')
visit

┌────────┬──────────────────────┬──────────────────────┬───┬──────────────┬────────────────┬──────────────────────┐
│ userId │       venueId        │   venueCategoryId    │ … │  longitude   │ timezoneOffset │     utcTimestamp     │
│ int64  │       varchar        │       varchar        │   │    double    │     int64      │       varchar        │
├────────┼──────────────────────┼──────────────────────┼───┼──────────────┼────────────────┼──────────────────────┤
│    470 │ 49bbd6c0f964a520f4…  │ 4bf58dd8d48988d127…  │ … │ -74.00258103 │           -240 │ Tue Apr 03 18:00:0…  │
│    979 │ 4a43c0aef964a520c6…  │ 4bf58dd8d48988d1df…  │ … │ -74.04416981 │           -240 │ Tue Apr 03 18:00:2…  │
│     69 │ 4c5cc7b485a1e21e00…  │ 4bf58dd8d48988d103…  │ … │ -73.88307006 │           -240 │ Tue Apr 03 18:02:2…  │
│    395 │ 4bc7086715a7ef3bef…  │ 4bf58dd8d48988d104…  │ … │ -73.98251878 │           -240 │ Tue Apr 03 18:02:4…  │
│     87 │ 4cf2c5321d18a14395…  │ 4bf58dd8d48988d1cb…  │ … │ -73.9896583

In [11]:
display(visit,3)

,userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,Tue Apr 03 18:00:09 +0000 2012
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.606800,-74.044170,-240,Tue Apr 03 18:00:25 +0000 2012
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.716162,-73.883070,-240,Tue Apr 03 18:02:24 +0000 2012


## Schéma d'une table

In [12]:
db.sql("""

SELECT column_name, data_type
FROM duckdb_columns
where table_name = 'Visit'

""")

┌─────────────────┬───────────┐
│   column_name   │ data_type │
│     varchar     │  varchar  │
├─────────────────┼───────────┤
│ userId          │ BIGINT    │
│ venueId         │ VARCHAR   │
│ venueCategoryId │ VARCHAR   │
│ venueCategory   │ VARCHAR   │
│ latitude        │ DOUBLE    │
│ longitude       │ DOUBLE    │
│ timezoneOffset  │ BIGINT    │
│ utcTimestamp    │ VARCHAR   │
└─────────────────┴───────────┘

# Requêtes

convertir le timestamp

In [13]:
visit2 = db.sql("""
  SELECT *, strptime(utcTimestamp, '%a %b %d %X %z %Y') as datetime
  FROM Visit
""")
visit2

┌────────┬──────────────────────┬───┬────────────────┬──────────────────────┬──────────────────────┐
│ userId │       venueId        │ … │ timezoneOffset │     utcTimestamp     │       datetime       │
│ int64  │       varchar        │   │     int64      │       varchar        │ timestamp with tim…  │
├────────┼──────────────────────┼───┼────────────────┼──────────────────────┼──────────────────────┤
│    470 │ 49bbd6c0f964a520f4…  │ … │           -240 │ Tue Apr 03 18:00:0…  │ 2012-04-03 18:00:0…  │
│    979 │ 4a43c0aef964a520c6…  │ … │           -240 │ Tue Apr 03 18:00:2…  │ 2012-04-03 18:00:2…  │
│     69 │ 4c5cc7b485a1e21e00…  │ … │           -240 │ Tue Apr 03 18:02:2…  │ 2012-04-03 18:02:2…  │
│    395 │ 4bc7086715a7ef3bef…  │ … │           -240 │ Tue Apr 03 18:02:4…  │ 2012-04-03 18:02:4…  │
│     87 │ 4cf2c5321d18a14395…  │ … │           -240 │ Tue Apr 03 18:03:0…  │ 2012-04-03 18:03:0…  │
│    484 │ 4b5b981bf964a52090…  │ … │           -240 │ Tue Apr 03 18:04:0…  │ 2012-04-03 18

In [14]:
visit3 = db.sql("""
  SELECT userId, venueId, datetime
  FROM visit2
""")

visit3

┌────────┬──────────────────────────┬──────────────────────────┐
│ userId │         venueId          │         datetime         │
│ int64  │         varchar          │ timestamp with time zone │
├────────┼──────────────────────────┼──────────────────────────┤
│    470 │ 49bbd6c0f964a520f4531fe3 │ 2012-04-03 18:00:09+00   │
│    979 │ 4a43c0aef964a520c6a61fe3 │ 2012-04-03 18:00:25+00   │
│     69 │ 4c5cc7b485a1e21e00d35711 │ 2012-04-03 18:02:24+00   │
│    395 │ 4bc7086715a7ef3bef9878da │ 2012-04-03 18:02:41+00   │
│     87 │ 4cf2c5321d18a143951b5cec │ 2012-04-03 18:03:00+00   │
│    484 │ 4b5b981bf964a520900929e3 │ 2012-04-03 18:04:00+00   │
│    642 │ 4ab966c3f964a5203c7f20e3 │ 2012-04-03 18:04:38+00   │
│    292 │ 4d0cc47f903d37041864bf55 │ 2012-04-03 18:04:42+00   │
│    428 │ 4ce1863bc4f6a35d8bd2db6c │ 2012-04-03 18:06:18+00   │
│    877 │ 4be319b321d5a59352311811 │ 2012-04-03 18:06:19+00   │
│     ·  │            ·             │           ·              │
│     ·  │            ·  

##  Ex1 Sequences de visites



### Numéro de POI

Définir les vues
 * venueId_poi(venueId, poi) qui associe le numéro original de venueId avec poi.
   * Indication, utiliser la fonction row_number() over( ...)
 * visit_poi(userId, poi, datetime) avec des numéros de poi allant de 1 à n



In [15]:
venueId_poi = db.sql("""
  select venueId, row_number() over(order by venueId) as poi
  from visit3
  group by venueId
""")



In [16]:
venueId_poi

┌──────────────────────────┬───────┐
│         venueId          │  poi  │
│         varchar          │ int64 │
├──────────────────────────┼───────┤
│ 3fd66200f964a52000e71ee3 │     1 │
│ 3fd66200f964a52000e81ee3 │     2 │
│ 3fd66200f964a52000f11ee3 │     3 │
│ 3fd66200f964a52001e51ee3 │     4 │
│ 3fd66200f964a52001e81ee3 │     5 │
│ 3fd66200f964a52002eb1ee3 │     6 │
│ 3fd66200f964a52003e51ee3 │     7 │
│ 3fd66200f964a52003e71ee3 │     8 │
│ 3fd66200f964a52003e81ee3 │     9 │
│ 3fd66200f964a52004e41ee3 │    10 │
│            ·             │     · │
│            ·             │     · │
│            ·             │     · │
│ 4b4f4d1bf964a5205a0027e3 │  9991 │
│ 4b4f593ef964a5207f0227e3 │  9992 │
│ 4b4f59def964a520a20227e3 │  9993 │
│ 4b4f5a31f964a520b20227e3 │  9994 │
│ 4b4f5dd8f964a520600327e3 │  9995 │
│ 4b4f601df964a520cf0327e3 │  9996 │
│ 4b4f6600f964a520cf0427e3 │  9997 │
│ 4b4f676cf964a520150527e3 │  9998 │
│ 4b4f676ff964a520160527e3 │  9999 │
│ 4b4f695ef964a520640527e3 │ 10000 │
├

In [20]:
visit_poi = db.sql("""
  select w.userId, v.poi, w.datetime
  from venueId_poi v, visit3 w
  where w.venueId ==  v.venueId
""")

visit_poi

┌────────┬───────┬──────────────────────────┐
│ userId │  poi  │         datetime         │
│ int64  │ int64 │ timestamp with time zone │
├────────┼───────┼──────────────────────────┤
│    688 │     1 │ 2013-02-16 02:29:11+00   │
│    953 │     2 │ 2013-01-07 22:30:07+00   │
│    602 │     3 │ 2013-02-03 04:19:08+00   │
│    558 │     4 │ 2012-05-16 03:24:17+00   │
│    352 │     5 │ 2013-01-25 23:45:38+00   │
│    122 │     6 │ 2013-02-08 02:49:02+00   │
│    110 │     7 │ 2012-07-04 03:49:33+00   │
│    349 │     8 │ 2013-02-07 01:15:05+00   │
│   1067 │     9 │ 2012-07-29 05:39:30+00   │
│    115 │    10 │ 2013-01-13 06:08:45+00   │
│     ·  │     · │           ·              │
│     ·  │     · │           ·              │
│     ·  │     · │           ·              │
│    236 │   948 │ 2012-08-18 02:47:22+00   │
│    586 │   951 │ 2012-11-11 16:40:39+00   │
│    636 │   952 │ 2012-12-15 05:13:35+00   │
│    365 │   955 │ 2012-06-16 21:51:12+00   │
│   1058 │   958 │ 2012-04-15 04:2

### 1.1) Rang
Pour chaque utilisateur, ordonner les visites par date et leur attribuer un **rang** allant de 1 (plus ancienne) à n (la plus récente)

In [21]:
visit_rank = db.sql("""
  select userId, poi, datetime, rank() over(partition by userid order by datetime) as rank
  from visit_poi
  order by userid
""")
visit_rank

┌────────┬───────┬──────────────────────────┬───────┐
│ userId │  poi  │         datetime         │ rank  │
│ int64  │ int64 │ timestamp with time zone │ int64 │
├────────┼───────┼──────────────────────────┼───────┤
│      1 │  6428 │ 2012-04-04 23:31:31+00   │     1 │
│      1 │ 24554 │ 2012-04-07 17:42:24+00   │     2 │
│      1 │ 26196 │ 2012-04-08 18:20:29+00   │     3 │
│      1 │  4131 │ 2012-04-08 20:02:10+00   │     4 │
│      1 │   963 │ 2012-04-09 16:20:52+00   │     5 │
│      1 │   525 │ 2012-04-10 00:24:31+00   │     6 │
│      1 │  1241 │ 2012-04-10 03:36:56+00   │     7 │
│      1 │ 33237 │ 2012-04-10 16:21:48+00   │     8 │
│      1 │  2643 │ 2012-04-12 17:19:21+00   │     9 │
│      1 │ 33237 │ 2012-04-13 15:41:41+00   │    10 │
│      · │    ·  │           ·              │     · │
│      · │    ·  │           ·              │     · │
│      · │    ·  │           ·              │     · │
│     56 │  5511 │ 2012-12-22 01:22:30+00   │   308 │
│     56 │  3681 │ 2012-12-2

### 1.2) Date précédente et déplacement
 On veut connaitre la durée de déplacement séparant deux visites consécutives d'un meme utilisateur.
Pour chaque visite d'un utilisateur, ajouter l'attribut **prev_date** contenant la date de la visite qu'il a effectuée précédemment.

Puis ajouter l'attribut **interval_duration** calculé par différence entre la date courante et la date précédente.
Indication, voir la fonction *date_diff('sec', a, b)*

In [22]:
visit_prev_date = db.sql("""
  select userId, poi, datetime, rank,
          min(datetime) over(partition by userid order by datetime rows between 1 preceding and current row) as prev_date
  from visit_rank
  order by userid
""")
visit_prev_date

┌────────┬───────┬──────────────────────────┬───────┬──────────────────────────┐
│ userId │  poi  │         datetime         │ rank  │        prev_date         │
│ int64  │ int64 │ timestamp with time zone │ int64 │ timestamp with time zone │
├────────┼───────┼──────────────────────────┼───────┼──────────────────────────┤
│      1 │  6428 │ 2012-04-04 23:31:31+00   │     1 │ 2012-04-04 23:31:31+00   │
│      1 │ 24554 │ 2012-04-07 17:42:24+00   │     2 │ 2012-04-04 23:31:31+00   │
│      1 │ 26196 │ 2012-04-08 18:20:29+00   │     3 │ 2012-04-07 17:42:24+00   │
│      1 │  4131 │ 2012-04-08 20:02:10+00   │     4 │ 2012-04-08 18:20:29+00   │
│      1 │   963 │ 2012-04-09 16:20:52+00   │     5 │ 2012-04-08 20:02:10+00   │
│      1 │   525 │ 2012-04-10 00:24:31+00   │     6 │ 2012-04-09 16:20:52+00   │
│      1 │  1241 │ 2012-04-10 03:36:56+00   │     7 │ 2012-04-10 00:24:31+00   │
│      1 │ 33237 │ 2012-04-10 16:21:48+00   │     8 │ 2012-04-10 03:36:56+00   │
│      1 │  2643 │ 2012-04-1

In [23]:
visit_duration = db.sql("""
  select userId, poi, datetime, rank, prev_date, date_diff('sec', prev_date, datetime) as interval_duration
  from visit_prev_date
""")

visit_duration

┌────────┬───────┬──────────────────────────┬───────┬──────────────────────────┬───────────────────┐
│ userId │  poi  │         datetime         │ rank  │        prev_date         │ interval_duration │
│ int64  │ int64 │ timestamp with time zone │ int64 │ timestamp with time zone │       int64       │
├────────┼───────┼──────────────────────────┼───────┼──────────────────────────┼───────────────────┤
│      1 │  6428 │ 2012-04-04 23:31:31+00   │     1 │ 2012-04-04 23:31:31+00   │                 0 │
│      1 │ 24554 │ 2012-04-07 17:42:24+00   │     2 │ 2012-04-04 23:31:31+00   │            238253 │
│      1 │ 26196 │ 2012-04-08 18:20:29+00   │     3 │ 2012-04-07 17:42:24+00   │             88685 │
│      1 │  4131 │ 2012-04-08 20:02:10+00   │     4 │ 2012-04-08 18:20:29+00   │              6101 │
│      1 │   963 │ 2012-04-09 16:20:52+00   │     5 │ 2012-04-08 20:02:10+00   │             73122 │
│      1 │   525 │ 2012-04-10 00:24:31+00   │     6 │ 2012-04-09 16:20:52+00   │           

### 1.3) Numéro de trajectoire

Séparer les visites de chaque utilisateur en trajectoires.

  * Dans une trajectoire, la durée entre deux visites consécutives ne peut pas dépasser 1 jour (soit 24h * 3600 secondes).

Ajouter l'attribut **traj** indiquant le numéro de la trajectoire pour un utilisateur donné.
Indication: on peut utiliser une expression *case when then else end*

In [24]:
visit_traj1 = db.sql("""
  select userid, poi, datetime, rank, prev_date, interval_duration,
          CASE
            WHEN interval_duration == 0 THEN 1
            WHEN interval_duration > 24*3600 THEN 1
            ELSE 0
          END as debut
  from visit_duration
""")
visit_traj1

┌────────┬───────┬──────────────────────────┬───────┬──────────────────────────┬───────────────────┬───────┐
│ userId │  poi  │         datetime         │ rank  │        prev_date         │ interval_duration │ debut │
│ int64  │ int64 │ timestamp with time zone │ int64 │ timestamp with time zone │       int64       │ int32 │
├────────┼───────┼──────────────────────────┼───────┼──────────────────────────┼───────────────────┼───────┤
│      1 │  6428 │ 2012-04-04 23:31:31+00   │     1 │ 2012-04-04 23:31:31+00   │                 0 │     1 │
│      1 │ 24554 │ 2012-04-07 17:42:24+00   │     2 │ 2012-04-04 23:31:31+00   │            238253 │     1 │
│      1 │ 26196 │ 2012-04-08 18:20:29+00   │     3 │ 2012-04-07 17:42:24+00   │             88685 │     1 │
│      1 │  4131 │ 2012-04-08 20:02:10+00   │     4 │ 2012-04-08 18:20:29+00   │              6101 │     0 │
│      1 │   963 │ 2012-04-09 16:20:52+00   │     5 │ 2012-04-08 20:02:10+00   │             73122 │     0 │
│      1 │   525 │ 

In [28]:
visit_traj2 = db.sql("""
    select userid, poi, datetime, rank, prev_date, interval_duration, debut,
            sum(debut)  over(partition by userid order by datetime rows between unbounded preceding and current row) as traj
    from visit_traj1
    order by userid
""")

visit_traj2

┌────────┬───────┬──────────────────────────┬───────┬──────────────────────────┬───────────────────┬───────┬────────┐
│ userId │  poi  │         datetime         │ rank  │        prev_date         │ interval_duration │ debut │  traj  │
│ int64  │ int64 │ timestamp with time zone │ int64 │ timestamp with time zone │       int64       │ int32 │ int128 │
├────────┼───────┼──────────────────────────┼───────┼──────────────────────────┼───────────────────┼───────┼────────┤
│      1 │  6428 │ 2012-04-04 23:31:31+00   │     1 │ 2012-04-04 23:31:31+00   │                 0 │     1 │      1 │
│      1 │ 24554 │ 2012-04-07 17:42:24+00   │     2 │ 2012-04-04 23:31:31+00   │            238253 │     1 │      2 │
│      1 │ 26196 │ 2012-04-08 18:20:29+00   │     3 │ 2012-04-07 17:42:24+00   │             88685 │     1 │      3 │
│      1 │  4131 │ 2012-04-08 20:02:10+00   │     4 │ 2012-04-08 18:20:29+00   │              6101 │     0 │      3 │
│      1 │   963 │ 2012-04-09 16:20:52+00   │     5 │ 20

## Ex2: Fenètres glissantes

### 2.1) Nombre de POI visités en une semaine

Ajouter l'attribut **nb_POI_7jours** donnant pour chaque jour où un utilisateur a fait des visites, le nombre de POI visités dans la semaine qui précède.

In [44]:
visit_7j = db.sql("""
    select userid, poi, datetime, rank, prev_date, interval_duration, debut, traj,
          count(poi) OVER seven as nb_POI_7j
    from visit_traj2
         WINDOW seven AS (
            PARTITION BY userid
            ORDER BY datetime
            RANGE BETWEEN INTERVAL 6 DAYS PRECEDING
                  AND CURRENT ROW
         )
    order by userid
""")

visit_7j

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────┬───────┬──────────────────────┬───────┬───┬───────────────────┬───────┬────────┬───────────┐
│ userId │  poi  │       datetime       │ rank  │ … │ interval_duration │ debut │  traj  │ nb_POI_7j │
│ int64  │ int64 │ timestamp with tim…  │ int64 │   │       int64       │ int32 │ int128 │   int64   │
├────────┼───────┼──────────────────────┼───────┼───┼───────────────────┼───────┼────────┼───────────┤
│      1 │  6428 │ 2012-04-04 23:31:3…  │     1 │ … │                 0 │     1 │      1 │         1 │
│      1 │ 24554 │ 2012-04-07 17:42:2…  │     2 │ … │            238253 │     1 │      2 │         2 │
│      1 │ 26196 │ 2012-04-08 18:20:2…  │     3 │ … │             88685 │     1 │      3 │         3 │
│      1 │  4131 │ 2012-04-08 20:02:1…  │     4 │ … │              6101 │     0 │      3 │         4 │
│      1 │   963 │ 2012-04-09 16:20:5…  │     5 │ … │             73122 │     0 │      3 │         5 │
│      1 │   525 │ 2012-04-10 00:24:3…  │     6 │ … │             29019 │

### 2.2) Nombre cumulé de visites par utilisateur

Pour chaque utilisateur ajouter un attribut **cumul_visites** indiquant le nombre de visites qu'il a effectuées

In [88]:
#visit_cumul_visites = db.sql("""
#    select userid, poi, datetime, rank, prev_date, interval_duration, debut, traj, nb_POI_7j,
#            count( distinct poi)  over(partition by userid  order by datetime range between  unbounded preceding and current row) as cumul_visites
#    from visit_7j
#    order by userid, datetime
#""")

#visit_cumul_visites

In [99]:
visit_first_seen = db.sql("""
    select userid, poi, datetime, rank, prev_date, interval_duration, debut, traj, nb_POI_7j,
      CASE
        WHEN 'ok' == (select 'ok' from visit_7j w  where w.userid = v.userid and w.poi = v.poi and w.datetime < v.datetime) THEN 0
        ELSE 1
      END as first_seen
    from visit_7j v
    order by userid, datetime
""")
visit_first_seen

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────┬───────┬──────────────────────┬───────┬──────────────────────┬───┬───────┬────────┬───────────┬────────────┐
│ userId │  poi  │       datetime       │ rank  │      prev_date       │ … │ debut │  traj  │ nb_POI_7j │ first_seen │
│ int64  │ int64 │ timestamp with tim…  │ int64 │ timestamp with tim…  │   │ int32 │ int128 │   int64   │   int32    │
├────────┼───────┼──────────────────────┼───────┼──────────────────────┼───┼───────┼────────┼───────────┼────────────┤
│      1 │  6428 │ 2012-04-04 23:31:3…  │     1 │ 2012-04-04 23:31:3…  │ … │     1 │      1 │         1 │          1 │
│      1 │ 24554 │ 2012-04-07 17:42:2…  │     2 │ 2012-04-04 23:31:3…  │ … │     1 │      2 │         2 │          1 │
│      1 │ 26196 │ 2012-04-08 18:20:2…  │     3 │ 2012-04-07 17:42:2…  │ … │     1 │      3 │         3 │          1 │
│      1 │  4131 │ 2012-04-08 20:02:1…  │     4 │ 2012-04-08 18:20:2…  │ … │     0 │      3 │         4 │          1 │
│      1 │   963 │ 2012-04-09 16:20:5…  │     5 

In [104]:
visit_cumul_visites = db.sql("""
    select userid, poi, datetime, rank, prev_date, interval_duration, debut, traj, nb_POI_7j,
    sum(first_seen) over(partition by userid order by datetime rows between unbounded preceding and current row) as cumul_visites
    from visit_first_seen
    order by userid, datetime
""")

visit_cumul_visites

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────┬───────┬──────────────────────┬───────┬───┬───────┬────────┬───────────┬───────────────┐
│ userId │  poi  │       datetime       │ rank  │ … │ debut │  traj  │ nb_POI_7j │ cumul_visites │
│ int64  │ int64 │ timestamp with tim…  │ int64 │   │ int32 │ int128 │   int64   │    int128     │
├────────┼───────┼──────────────────────┼───────┼───┼───────┼────────┼───────────┼───────────────┤
│      1 │  6428 │ 2012-04-04 23:31:3…  │     1 │ … │     1 │      1 │         1 │             1 │
│      1 │ 24554 │ 2012-04-07 17:42:2…  │     2 │ … │     1 │      2 │         2 │             2 │
│      1 │ 26196 │ 2012-04-08 18:20:2…  │     3 │ … │     1 │      3 │         3 │             3 │
│      1 │  4131 │ 2012-04-08 20:02:1…  │     4 │ … │     0 │      3 │         4 │             4 │
│      1 │   963 │ 2012-04-09 16:20:5…  │     5 │ … │     0 │      3 │         5 │             5 │
│      1 │   525 │ 2012-04-10 00:24:3…  │     6 │ … │     0 │      3 │         6 │             6 │
│      1 │

## Exercice 3

### 3.1) Visites dans une trajectoire
On constate qu'une trajectoire peut contenir plusieurs visites **consécutives** d'un même POI et on veut "fusionner" ces visites.
Pour chaque trajectoire, agréger les visites consécutives d'un meme POI. Définir les attributs date_début, date_fin pour la visite d'un POI.

Rmq1: s'il n'y a pas plusieurs visites consécutives pour un POI alors ses dates de début et de fin sont identiques.

Rmq2: une trajectoire peut contenir plusieurs visites non consécutives d'un même POI.

afficher les visites d'une trajectoire pour laqelle au moins une fusion a été faite.

<font color='blue'> Tout d'abord on cherche les séquences consécutives de POI dans une même trajectoire que l'on marque par 1 dans la colonne fusionne. </font>


In [167]:


visit_traj_fus = db.sql("""
    select userid, poi, interval_duration, datetime, traj,
      CASE
        WHEN poi == lag(poi) over(PARTITION BY userid, traj order by datetime ) THEN 1
        WHEN poi == lead(poi) over(PARTITION BY userid, traj order by datetime ) THEN 1
        ELSE 0
      END as fusionne
    from visit_traj2
    order by userid, datetime
""")

visit_traj_fus

┌────────┬───────┬───────────────────┬──────────────────────────┬────────┬──────────┐
│ userId │  poi  │ interval_duration │         datetime         │  traj  │ fusionne │
│ int64  │ int64 │       int64       │ timestamp with time zone │ int128 │  int32   │
├────────┼───────┼───────────────────┼──────────────────────────┼────────┼──────────┤
│      1 │  6428 │                 0 │ 2012-04-04 23:31:31+00   │      1 │        0 │
│      1 │ 24554 │            238253 │ 2012-04-07 17:42:24+00   │      2 │        0 │
│      1 │ 26196 │             88685 │ 2012-04-08 18:20:29+00   │      3 │        0 │
│      1 │  4131 │              6101 │ 2012-04-08 20:02:10+00   │      3 │        0 │
│      1 │   963 │             73122 │ 2012-04-09 16:20:52+00   │      3 │        0 │
│      1 │   525 │             29019 │ 2012-04-10 00:24:31+00   │      3 │        0 │
│      1 │  1241 │             11545 │ 2012-04-10 03:36:56+00   │      3 │        0 │
│      1 │ 33237 │             45892 │ 2012-04-10 16:2

<font color='blue'> On peut désormais connaître les numéros des trajectoire pour chaque userid tels qu'une fusion soit nécessaire. On affiche ces couples ci-dessous. </font>


In [168]:
visit_traj_fus_aff = db.sql("""
    select userid, traj
    from visit_traj_fus
    where fusionne == 1
    group by userid, traj
""")
visit_traj_fus_aff

┌────────┬────────┐
│ userId │  traj  │
│ int64  │ int128 │
├────────┼────────┤
│      1 │      4 │
│      4 │      2 │
│      5 │      2 │
│      5 │      4 │
│      5 │      5 │
│      5 │      8 │
│      7 │      1 │
│      7 │      5 │
│      7 │     18 │
│      7 │     27 │
│      · │      · │
│      · │      · │
│      · │      · │
│    971 │     14 │
│    995 │      5 │
│   1000 │     14 │
│   1015 │     18 │
│   1042 │     12 │
│    974 │      9 │
│    959 │      1 │
│    965 │     21 │
│    996 │     14 │
│    993 │     37 │
├────────┴────────┤
│    4329 rows    │
│   (20 shown)    │
└─────────────────┘

<font color='blue'> On affiche les trajectoires pour lesquelles une fusion sera necessaire. On se limite à l'affiche des 20 premiers n-uplets ci-dessous :   </font>


In [169]:
visit_traj_consec_aff2 = db.sql("""
  select v.userid, v.poi, v.traj, v.datetime
  from visit_traj2 v, visit_traj_fus_aff a
  where v.userid==a.userid and v.traj==a.traj
  order by userid
  limit 20
""")
visit_traj_consec_aff2

┌────────┬───────┬────────┬──────────────────────────┐
│ userId │  poi  │  traj  │         datetime         │
│ int64  │ int64 │ int128 │ timestamp with time zone │
├────────┼───────┼────────┼──────────────────────────┤
│      1 │  2643 │      4 │ 2012-04-12 17:19:21+00   │
│      1 │ 33237 │      4 │ 2012-04-13 15:41:41+00   │
│      1 │  2150 │      4 │ 2012-04-14 01:11:20+00   │
│      1 │ 23851 │      4 │ 2012-04-14 03:07:56+00   │
│      1 │   983 │      4 │ 2012-04-14 04:45:13+00   │
│      1 │  2522 │      4 │ 2012-04-14 15:57:20+00   │
│      1 │  1309 │      4 │ 2012-04-14 17:45:23+00   │
│      1 │ 12421 │      4 │ 2012-04-14 20:03:42+00   │
│      1 │  7949 │      4 │ 2012-04-14 21:24:08+00   │
│      1 │ 30004 │      4 │ 2012-04-14 22:07:43+00   │
│      1 │   842 │      4 │ 2012-04-14 23:15:07+00   │
│      1 │   842 │      4 │ 2012-04-14 23:15:50+00   │
│      4 │  9323 │      2 │ 2012-04-07 17:45:24+00   │
│      4 │  9323 │      2 │ 2012-04-08 12:23:38+00   │
│      5 │

<font color='blue'> Enfin, on effectue la fusion des POI consécutif en prenant la date de début et la date de fin. On fait l'union avec les n-uplets pour lesquels aucune fusion n'était necessaire (pour lesquels ces deux dates sont identiques). </font>


In [172]:
visit_traj_consec = db.sql("""
  select userid, poi, traj, MIN(datetime) as date_debut, MAX(datetime) as date_fin
  from visit_traj_fus
  where fusionne = 1
  group by userid, poi, traj
  UNION
  select userid, poi, traj, datetime as date_debut, datetime as date_fin
  from visit_traj_fus
  where fusionne = 0
  order by userid, date_debut
""")
visit_traj_consec

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────┬───────┬────────┬──────────────────────────┬──────────────────────────┐
│ userId │  poi  │  traj  │        date_debut        │         date_fin         │
│ int64  │ int64 │ int128 │ timestamp with time zone │ timestamp with time zone │
├────────┼───────┼────────┼──────────────────────────┼──────────────────────────┤
│      1 │  6428 │      1 │ 2012-04-04 23:31:31+00   │ 2012-04-04 23:31:31+00   │
│      1 │ 24554 │      2 │ 2012-04-07 17:42:24+00   │ 2012-04-07 17:42:24+00   │
│      1 │ 26196 │      3 │ 2012-04-08 18:20:29+00   │ 2012-04-08 18:20:29+00   │
│      1 │  4131 │      3 │ 2012-04-08 20:02:10+00   │ 2012-04-08 20:02:10+00   │
│      1 │   963 │      3 │ 2012-04-09 16:20:52+00   │ 2012-04-09 16:20:52+00   │
│      1 │   525 │      3 │ 2012-04-10 00:24:31+00   │ 2012-04-10 00:24:31+00   │
│      1 │  1241 │      3 │ 2012-04-10 03:36:56+00   │ 2012-04-10 03:36:56+00   │
│      1 │ 33237 │      3 │ 2012-04-10 16:21:48+00   │ 2012-04-10 16:21:48+00   │
│      1 │  2643

<font color='blue'> On affiche à nouveau les 20 premier n-uplet des trajectoire pour lesquelles une fusion a été nécessaire et on observe bien que la fusion a été correctement effectuée (par exemple pour le POI 842 pour l'utilisateur 1). </font>

In [173]:
visit_traj_consec_fusion_aff = db.sql("""
  select v.userid, v.poi, v.traj, v.date_debut, v.date_fin
  from visit_traj_consec v, visit_traj_fus_aff a
  where v.userid==a.userid and v.traj==a.traj
  order by userid
  limit 20
""")
visit_traj_consec_fusion_aff

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────┬───────┬────────┬──────────────────────────┬──────────────────────────┐
│ userId │  poi  │  traj  │        date_debut        │         date_fin         │
│ int64  │ int64 │ int128 │ timestamp with time zone │ timestamp with time zone │
├────────┼───────┼────────┼──────────────────────────┼──────────────────────────┤
│      1 │  2643 │      4 │ 2012-04-12 17:19:21+00   │ 2012-04-12 17:19:21+00   │
│      1 │ 33237 │      4 │ 2012-04-13 15:41:41+00   │ 2012-04-13 15:41:41+00   │
│      1 │  2150 │      4 │ 2012-04-14 01:11:20+00   │ 2012-04-14 01:11:20+00   │
│      1 │ 23851 │      4 │ 2012-04-14 03:07:56+00   │ 2012-04-14 03:07:56+00   │
│      1 │   983 │      4 │ 2012-04-14 04:45:13+00   │ 2012-04-14 04:45:13+00   │
│      1 │  2522 │      4 │ 2012-04-14 15:57:20+00   │ 2012-04-14 15:57:20+00   │
│      1 │  1309 │      4 │ 2012-04-14 17:45:23+00   │ 2012-04-14 17:45:23+00   │
│      1 │ 12421 │      4 │ 2012-04-14 20:03:42+00   │ 2012-04-14 20:03:42+00   │
│      1 │  7949